In [54]:
import torch
import argparse
import os
import numpy as np

# TensorBoard
from torch.utils.tensorboard import SummaryWriter

from data import get_dataset
from experiment import ex
from model import load_model

from utils import post_config_hook
from utils.yaml_config_hook import yaml_config_hook
from utils.eval import eval_all
from utils.youtube import download_yt
from datasets.utils.resample import convert_samplerate

import matplotlib.pyplot as plt


cfg = yaml_config_hook("./config/config.yaml")

args = argparse.Namespace(**cfg)
args.dataset = "msd"
args.model_path = "/storage/jspijkervet/logs_backup_ws7/clmr/2/"
args.epoch_num= 1490
args.finetune_model_path = "/storage/jspijkervet/logs_backup_ws7/clmr/4"
args.finetune_epoch_num = 50
args.n_classes = 50 # 50 tags

In [55]:
args.lin_eval = True
args.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
args.batch_size = args.logistic_batch_size

In [56]:
(train_loader, train_dataset, val_loader, val_dataset, test_loader, test_dataset) = get_dataset(args)

201676it [00:05, 33748.23it/s]


Num segments: 2016760
Num tracks: 201676
[Train dataset (msd_22050)]


11774it [00:00, 41108.85it/s]


Num segments: 117740
Num tracks: 11774
[Validation dataset (msd_22050)]


28434it [00:01, 20822.59it/s]


Num segments: 284340
Num tracks: 28434
[Test dataset (msd_22050)]


In [62]:
train_labels = np.array([l[2].numpy() for l in train_dataset.tracks_list_test])
test_labels = np.array([l[2].numpy() for l in test_dataset.tracks_list_test])

In [66]:
for k, train, test in zip(train_dataset.tags, train_labels.sum(axis=0), test_labels.sum(axis=0)):
    print(k, train, test)

rock 39650.0 4232.0
pop 21726.0 3041.0
alternative 14906.0 1686.0
indie 19197.0 2709.0
electronic 17082.0 2605.0
female vocalists 14937.0 1928.0
dance 8627.0 1456.0
00s 3018.0 392.0
alternative rock 7680.0 784.0
jazz 13788.0 2564.0
beautiful 2987.0 331.0
metal 6426.0 1013.0
chillout 5470.0 757.0
male vocalists 2510.0 359.0
classic rock 8369.0 686.0
soul 10075.0 1354.0
indie rock 7694.0 988.0
Mellow 3019.0 407.0
electronica 4864.0 721.0
80s 7865.0 1032.0
folk 8929.0 1266.0
90s 4462.0 547.0
chill 2086.0 298.0
instrumental 6885.0 1079.0
punk 8774.0 807.0
oldies 5512.0 981.0
blues 9151.0 995.0
hard rock 6291.0 496.0
ambient 5274.0 855.0
acoustic 4653.0 572.0
experimental 4786.0 747.0
female vocalist 1840.0 244.0
guitar 2790.0 351.0
Hip-Hop 7541.0 1230.0
70s 4143.0 492.0
party 1015.0 168.0
country 8215.0 1165.0
easy listening 1987.0 258.0
sexy 1153.0 200.0
catchy 1076.0 131.0
funk 4409.0 688.0
electro 3351.0 680.0
heavy metal 3596.0 519.0
Progressive rock 4694.0 963.0
60s 4604.0 639.0
rnb 4